In [ ]:
import tensorflow as tf
import numpy as np
import os
from tqdm import tqdm

2025-10-07 19:53:44.882357: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-07 19:53:44.893796: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-07 19:53:44.908449: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8473] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-07 19:53:44.915414: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1471] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-07 19:53:44.931325: I tensorflow/core/platform/cpu_feature_guar

In [ ]:
XVIEW_RECOGNITION_PATH = "../data/xview_recognition"

import numpy as np
import json
import os
import uuid
import numpy as np

class GenericObject:
    """
    Generic object data.
    """
    def __init__(self):
        self.id = uuid.uuid4()
        self.bb = (-1, -1, -1, -1)
        self.category= -1
        self.score = -1

class GenericImage:
    """
    Generic image data.
    """
    def __init__(self, filename):
        self.filename = filename
        self.tile = np.array([-1, -1, -1, -1])  # (pt_x, pt_y, pt_x+width, pt_y+height)
        self.objects = list([])

    def add_object(self, obj: GenericObject):
        self.objects.append(obj)

json_file = os.path.join(XVIEW_RECOGNITION_PATH, 'xview_ann_train.json')
with open(json_file) as ifs:
    json_data = json.load(ifs)
ifs.close()

categories = {0: 'Cargo plane', 1: 'Small car', 2: 'Bus', 3: 'Truck', 4: 'Motorboat', 5: 'Fishing vessel', 6: 'Dump truck', 7: 'Excavator', 8: 'Building', 9: 'Helipad', 10: 'Storage tank', 11: 'Shipping container', 12: 'Pylon'}

id_to_name = {v: k for k, v in categories.items()}
counts = {name: 0 for name in categories.values()}

anns = []
for json_img, json_ann in zip(json_data['images'].values(), json_data['annotations'].values()):
    image = GenericImage(json_img['filename'])
    image.tile = np.array([0, 0, json_img['width'], json_img['height']])
    obj = GenericObject()
    obj.bb = (int(json_ann['bbox'][0]), int(json_ann['bbox'][1]), int(json_ann['bbox'][2]), int(json_ann['bbox'][3]))
    
    category_name = json_ann['category_id']
    if category_name != "Unknown":
        counts[category_name] += 1
    obj.category = id_to_name[category_name]

    image.add_object(obj)
    anns.append(image)

print("Distribución de categorías en el dataset:")
print(counts)

from sklearn.model_selection import train_test_split

anns_train, anns_valid = train_test_split(anns, test_size=0.1, random_state=1, shuffle=True)
print('Number of training images: ' + str(len(anns_train)))
print('Number of validation images: ' + str(len(anns_valid)))

objs_train = [(ann.filename, obj) for ann in anns_train for obj in ann.objects]
objs_valid = [(ann.filename, obj) for ann in anns_valid for obj in ann.objects]

Distribución de categorías en el dataset:
{'Cargo plane': 635, 'Small car': 3324, 'Bus': 1768, 'Truck': 2210, 'Motorboat': 1069, 'Fishing vessel': 706, 'Dump truck': 1236, 'Excavator': 789, 'Building': 3594, 'Helipad': 111, 'Storage tank': 1469, 'Shipping container': 1523, 'Pylon': 312}
Number of training images: 16871
Number of validation images: 1875


In [11]:
import warnings
import rasterio
import numpy as np

def load_geoimage(filename_tensor):
    warnings.filterwarnings('ignore', category=rasterio.errors.NotGeoreferencedWarning)
    filename = filename_tensor
    src_raster = rasterio.open(filename, 'r')

    input_type = src_raster.profile['dtype']
    input_channels = src_raster.count
    img = np.zeros((src_raster.height, src_raster.width, src_raster.count), dtype=input_type)
    for band in range(input_channels):
        img[:, :, band] = src_raster.read(band+1)
    return img

In [ ]:
def _bytes_feature(value):
    """Devuelve un bytes_list a partir de un string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Devuelve un int64_list a partir de un bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(image, category_id):
    """
    Crea un tf.train.Example a partir de los datos de una imagen.
    """
    image_bytes = image.tobytes()
    
    feature = {
        'height': _int64_feature(image.shape[0]),
        'width': _int64_feature(image.shape[1]),
        'depth': _int64_feature(image.shape[2]),
        'category_id': _int64_feature(category_id),
        'image_raw': _bytes_feature(image_bytes),
    }
    
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [ ]:
def convert_to_tfrecord(objs, output_filename):
    print(f"Escribiendo {output_filename}...")
    with tf.io.TFRecordWriter(output_filename) as writer:
        for filename, obj in tqdm(objs):
            image_path = os.path.abspath(os.path.join(XVIEW_RECOGNITION_PATH, filename))
            image_np = load_geoimage(image_path)
            category_id = obj.category
            
            serialized_example = serialize_example(image_np, category_id)
            writer.write(serialized_example)

In [ ]:
# --- Ejecutar la conversión ---

convert_to_tfrecord(objs_train, 'train.tfrecord')
convert_to_tfrecord(objs_valid, 'validation.tfrecord')

print("¡Conversión completada!")

Escribiendo train.tfrecord...


  0%|          | 0/16871 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1759869269.733458    8208 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1759869269.871026    8208 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1759869269.871065    8208 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1759869269.876260    8208 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1759869269.876318    8208 cuda_executor.cc:1001] could not open file

Escribiendo validation.tfrecord...


100%|██████████| 1875/1875 [01:38<00:00, 18.96it/s] 

¡Conversión completada!
